In [2]:
import fla_parallelized as a
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import pickle as pkl
from importlib import reload
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import math
import time
import os
import pickle as pkl
# from torch.utils.tensorboard import SummaryWriter
import utils as u
import matplotlib.pyplot as plt

In [3]:
cwd=os.getcwd()
with open(f"{cwd}/data/diabetes/X.pkl", "rb") as file:
    X_raw = pkl.load(file)
with open(f"{cwd}/data/diabetes/y.pkl", "rb") as file:
    y = pkl.load(file)
y_counts = np.unique(y, return_counts=True)[1]
weight = torch.tensor([y_counts[0]/y_counts[1]], dtype=torch.float32)

In [4]:
imputer = KNNImputer(n_neighbors=5)
X_imputed_not_norm = imputer.fit_transform(X_raw)
scaler = MinMaxScaler()
X = scaler.fit_transform(X_imputed_not_norm)

In [11]:
reload(a)
seeds = 10
test_predictions_seed = [[]]*seeds
test_label_list_seed = [[]]*seeds
losses_seed = []
activations = [nn.ReLU, nn.Sigmoid, a.GaussianActivation]
forward_times = []
for seed in range(seeds):
    print(f'seed {seed+1}')
    #split data
    X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=seed)
    X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, stratify=y_train, test_size=0.1, random_state=seed)
    train_dataset = a.npDataset(X_train,y_train)
    test_dataset = a.npDataset(X_test,y_test)
    val_dataset = a.npDataset(X_val,y_val)
    batch_size = 64
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = nn.Sequential(
        a.MultiActivationLinear(input_dim=108, activations=activations,nodes_per_activation=17),
        a.MultiActivationLinear(input_dim=17*3, activations=activations,nodes_per_activation=8),
        a.MultiActivationLinear(input_dim=8*3, activations=activations,nodes_per_activation=3),
        nn.Linear(in_features=3*3,out_features=1)
    )
    criterion = nn.BCEWithLogitsLoss(pos_weight=weight)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    #train
    num_epochs = 500
    best_val_loss = float('inf')
    best_model = None
    patience = 10
    early_stop_counter = 0
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_losses = []
        for inputs, labels in val_loader:
            with torch.no_grad():
                outputs = model(inputs)
                labels = labels.unsqueeze(1)
                val_loss = criterion(outputs, labels)
                val_losses.append(val_loss.item())
        
        avg_val_loss = np.mean(val_losses)
        print(f'Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}')
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model = model.state_dict()
            early_stop_counter = 0
        else:
            early_stop_counter += 1
        
        if early_stop_counter >= patience:
            print(f'Early stopping after epoch {epoch+1} with validation loss {best_val_loss:.4f}')
            break
        
    model.load_state_dict(best_model)

    #eval
    test_losses = []
    test_predictions = []
    test_true_labels = []

    for inputs, labels in test_loader:
        with torch.no_grad():
            outputs = model(inputs)
            labels = labels.unsqueeze(1)
            test_loss = criterion(outputs, labels)
            test_losses.append(test_loss.item())
            test_predictions.extend(outputs.cpu().numpy())
            test_true_labels.extend(labels.cpu().numpy())
    avg_test_loss = np.mean(test_losses)
    test_predictions_f1 = [F.sigmoid(torch.tensor(y))>0.5 for y in test_predictions]
    test_score = f1_score(test_true_labels, test_predictions_f1)
    print(f'Test Loss: {avg_test_loss:.4f}, Test Score: {test_score:.4f} for seed {seed+1}')
    test_label_list_seed[seed].append(test_true_labels)
    test_predictions_seed[seed].append(test_predictions)
    losses_seed.append(avg_test_loss)

seed 1
Epoch 1, Validation Loss: 1.1436
Epoch 2, Validation Loss: 1.1112
Epoch 3, Validation Loss: 1.1171
Epoch 4, Validation Loss: 1.1244
Epoch 5, Validation Loss: 1.1294
Epoch 6, Validation Loss: 1.1555
Epoch 7, Validation Loss: 1.1717
Epoch 8, Validation Loss: 1.1922
Epoch 9, Validation Loss: 1.2582
Epoch 10, Validation Loss: 1.2294
Epoch 11, Validation Loss: 1.3149
Epoch 12, Validation Loss: 1.3350
Early stopping after epoch 12 with validation loss 1.1112
Test Loss: 1.4854, Test Score: 0.2169 for seed 1
seed 2
Epoch 1, Validation Loss: 1.1507
Epoch 2, Validation Loss: 1.1248
Epoch 3, Validation Loss: 1.1236
Epoch 4, Validation Loss: 1.1516
Epoch 5, Validation Loss: 1.2134
Epoch 6, Validation Loss: 1.2227
Epoch 7, Validation Loss: 1.2607
Epoch 8, Validation Loss: 1.3477
Epoch 9, Validation Loss: 1.4526
Epoch 10, Validation Loss: 1.6279
Epoch 11, Validation Loss: 1.4567
Epoch 12, Validation Loss: 1.6693
Epoch 13, Validation Loss: 1.8569
Early stopping after epoch 13 with validation l

In [12]:
print(np.median(losses_seed))

1.6067627109587193
